<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [1]:
!pip install pyspark
!pip install findspark

In [ ]:
# Start session

In [5]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("capstone project").getOrCreate()

23/12/19 16:27:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv


--2023-12-19 16:25:38--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2023-12-19 16:25:38 (43.7 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [ ]:
# Load the csv into a spark dataframe

In [6]:
df = spark.read.csv('searchterms.csv',header=True,inferSchema=True)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [11]:
rows = df.count()
cols = len(df.columns)
print("Number of columns: {}".format(cols))
print("rows: ", rows)

Number of columns: 4
rows:  10000


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [12]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [20]:
datatypes = df.dtypes
print (datatypes[-1][1])

string


In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [23]:
from pyspark.sql.functions import col
search_count = df.filter(col('searchterm') == 'gaming laptop').count()
print(search_count)

499


In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [35]:
search_counts = df.groupBy('searchterm').count()
sorted_search_counts = search_counts.orderBy(col("count").desc())
sorted_search_counts.show(5)

[Stage 18:===================================================>  (192 + 8) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xvzf model.tar.gz

In [47]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load('sales_prediction.model')


In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [48]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year, 0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features', 'year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [49]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/12/19 16:54:13 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/12/19 16:54:13 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
